# **Introduction**

**<center><h1>KERAS.LOSSES</h1></center>**<br>
Dans keras.losses, on a des fonctions et des classes:<br>
1. **keras.losses.categorical_crossentropy(y_true, y_pred, from_logits=True)** est une fonction qu'on directement utilisée dans tf.GradientTape.<br>
2. **a = keras.losses.CategoricalCrossentropy(from_logits=True)** est une classe, on fait **a(y_true, y_pred)**.<br>
On a aussi **KERAS.METRICS**, **KERAS.OPTIMIZERS**, ETC.<br>

**<center><h1>Training Keras Model</h1></center>**<br>
On peut noter **04** approches:<br>
1. **fit:** c'est la méthode standard.<br>

2. **keras.Model:** Créer une classe qui hérite de la classe **keras.models.Model** or **keras.Model**. Cette classe doit implémenter **train_step** qui utilise **tf.GradientTape** pour mettre à jour les **poids** du modèle par **batch**. On peut aussi implémenter **test_step** pour le test, **call** pour le **forward**, **metrics** qui retourne la liste des métriques (mise à jour), **computer_loss** pour retourner la **loss**.<br>

3. **@tf.function:** Créer une fonction en la faisant précédée par **@tf.function**. Cette approche permet à la fonction créée de pouvoir être compiler c'est à dire utiliser le **tf.GradientTape**. La fonction ainsi créée peut être utilisée dans une fonction qui fait une boucle sur le nombre d'époques et donne entrée **batch** par **batch** à la fonction qui met à jour les poids suivant les batchs donnés.<br>

4. **train_on_batch:** C'est une **méthode** de la classe Model de keras, tout modèle de keras peut donc y accéder. On crée d'abord un modèle **model**, on utilise d'abord une boucle sur le nombre d'époque suivie d'une boucle qui prend **batch par batch**, on fait à chaque fois **model.train_on_batch(x_batch, y_batch)** qui retourne la **loss** sur le batch et les métriques si c'es définit dans le compile de **model**.<br>

**<center><h1>Passage par valeur des modèles</h1></center>**<br>
Supposons qu'on ait 03 modèles: <br>

**model1 = tf.keras.models.Sequential()**<br>
**model2 = tf.keras.models.Sequential()**<br>
**model3 = tf.keras.models.Sequential()**<br>

**model3.add(model1**)<br>
**mdoel3.model1.trainable = False** <br>
**model3.add(model2)**<br>

model1 est utilisé dans model3, même si c'est son trainable=False, toute modification (mise à jour) fait sur model1 en dehors va se répercuter sur model3.model1, même si les modifications sont faites à l'intérieur d'une fonction.<br>
**NB:** Dès qu'on crée un modèle, on l'utilise pour intialiser ou l'affecter ou à un autre modèle les modifications apportées sur un de ces modèles affectent le modèle de base. Voir la section: Method using train_on_batch.<br>

**<center><h1>Bahdanau Attention</h1></center>**<br>
C'est une attention additive, l'objectif est de prendre l'état caché du decoder comme query et le output de l'encoder comme value pour produre un vecteur de contexte. Le vecteur de contexte est concaténé avec l'embedding de l'entrée du decoder (qui prend token par token), et sert d'entrée à la couche rnn, lstm ou gru.<br>

Q = Decoder hidden state (None, decoder_unit)<br>
    at the begining, Q = Encoder hidden state<br>

V = Encoder output (None, sequence, feature)<br>

Q = tf.expand_dims(Q, axis=1) # (None, 1, decoder_unit)<br>
EncQ = Dense1(Q) # (None, 1, units)<br>
EncV = Dense2(V) # (None, sequence, units)<br>
sum = tf.nn.tanh(EncQ+EncV) # activation (None, sequence, units)<br>

score = Dense3(sum) # (None, sequence, 1)<br>
attention = tf.nn.softmax(score, axis=1) # (None, sequence, 1) softmax based on token position<br>

context_vector = attention*V # (None, sequence, feature)<br>
context_vector = tf.nn.sum(context_vector, axis=1) # (None, feature)<br>

context_vector sera utilisé dans le decoder et concaténer avec l'entrée (qui se fait token par token)<br>
x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1) # (None, 1, feature+embedding_dim_x) x vient de l'embedding x est de type (None, 1) car on traite par token Embedding donne (None, 1, embedding_dim_x)<br>


**<center><h1>Paramètres et Sorties</h1></center>**<br>

**<h2>MLP (Dense)</h2>**<br>
**1. Sortie:** Si l'entrée est de dimension 2, la sortie sera **(None, units)**, si l'entrée est de dimension 3 (séquences), la sortie sera **(None, sequence, units)**, si l'entrée est de dimension 4 (convolutions ou images), la sortie sera **(None, H, W, units)**.<br>

**2. Paramètres:** **D*W + W**, avec D la dimension de l'entrée, W le nombre de units.<br>

**<h2>CNN (Conv2D)</h2>**<br>
**1. Sortie:** **math.floor((n - f + p) / s) + 1**, avec n la taille de l'entrée, f la taille du filtre, p = f - 1 si padding = 'same' et p = 0 si padding='valid', s est la taille du stride. ON considère la partie entière inférieure.<br>
**NB:** Si conv2DTranspose ou UpSampling2D, on effectue l'opération inverse **s × (n + f - p)**.<br>


**Paramètres:** Le nombre de paramètres de la couche, dépend du nombre de canal de la couche précédente. La couche actuelle contient par exemple 2 filtres, et que la couche précédente contient 3 canaux, le filtre 1 va être dupliqué sur les 3 canaux et si la taille du filtre est (2, 2), donc y aura 3 × (2 × 2) paramètres à apprendre avec le même biais donc 3 × (2 × 2) + 1. Le même processus s'applique sur le filtre 2. La formule générale est alors:<br> <center>**(hauteur_filtre × largeur_filtre × nombre_canaux_couche_precedente + 1 ) × nombre_filtres**</center><br>

**Les couches de Pooling n'ont pas de paramètres à apprendre**<br>


**<h2>RNN</h2>**<br>
**1. Sortie:** Par défaut, retourne (None, units). Si return_sequences = True, (None, sequence, units). Si return_state = True, on aura deux outputs, (None, units), (None, units). Si return_sequences=True et return_state=True on aura (None, sequence, units), (None, units).<br>
**NB:** Ceci est valable pour les **GRU**, pour les **LSTM**, return_sate=True donne deux état (None, units) pour hidden_state et (None, units) pour cell_state.

**2. Paramètres:** L'entrée correspond à deux vecteurs concaténés: x (dernière dimension de l'entrée: **shape[-1]**) et h. x le token d'entrée et h l'état caché.<br>
**<center>RNN: (x + h) × h + h</center>**<br>
**<center>LSTM: 4 × [(x + h) × h + h] </center>** <br>
**<center>GRU: 3 × [(x + h) × h + h]</center>**

**Calcul de AP (Average Precision) et mAP (mean Average Precision)**<br>
https://www.youtube.com/watch?v=FppOzcDvaDI&list=PL3Q9L9bKi20IatsPIesEJeu6JYBY-Wk1G&index=3&t=394s

In [ ]:
# Random projection: https://scikit-learn.org/stable/modules/random_projection.html

In [ ]:
"""
1. loss = tf.losses.BinaryCrossEntropy(): loss(y_true, y_pred) fait la somme et retourne une seule valeur
   Pour que loss soit une liste, il faut mettre reduction = 'none' sur la classe BinaryCrossEtropy(reduction='none')

2. loss = tf.losses.binary_crossentropy(y_true, y_pred) retourne une liste de loss
"""

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/GradientTape
# GradientTape permet d'effectuer une opération différentielle
import tensorflow as tf

x = tf.constant(3.0)
with tf.GradientTape() as g:
  g.watch(x)
  y = x * x
dy_dx = g.gradient(y, x)
print(dy_dx)

In [ ]:
x = tf.constant(5.0)
with tf.GradientTape() as g:
  g.watch(x)
  with tf.GradientTape() as gg:
    gg.watch(x)
    y = x * x
  dy_dx = gg.gradient(y, x)  # dy_dx = 2 * x
d2y_dx2 = g.gradient(dy_dx, x)  # d2y_dx2 = 2


print(dy_dx)
print(d2y_dx2)


In [ ]:
# Persitent = True permet d'appeller le tape plusieurs fois

x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as g:
  g.watch(x)
  y = x * x
  z = y * y
dz_dx = g.gradient(z, x)  # (4*x^3 at x = 3)
print(dz_dx)

dy_dx = g.gradient(y, x)
print(dy_dx)

"""
NB: si on utilise un modèle sans watch, par défaut toutes les trainable_variables sont mis dans watch de manière implicite, donc on a pas besoin de le spéficier
    car les poids sont des tf.Variable pas des tf.constant, et par défaut un tf.Variable est directement mis dans watch.
    Si on veut enléver cet effet par défaut, on utilise le paramètre watch_accessed_variables=False (voir exemple suivant)
"""

In [ ]:
x = tf.Variable(2.0) # watch actif par défaut
w = tf.Variable(5.0) # watch actif par défaut
with tf.GradientTape(
    watch_accessed_variables=False, persistent=True) as tape:
  tape.watch(x)
  y = x ** 2  # Gradients will be available for `x`.
  z = w ** 3  # No gradients will be available as `w` isn't being watched.
dy_dx = tape.gradient(y, x)
print(dy_dx) # tf.Tensor(4.0, shape=(), dtype=tf.float32)

# No gradients will be available as `w` isn't being watched.
dz_dw = tape.gradient(z, w)
print(dz_dw) # None

In [ ]:
a = tf.keras.layers.Dense(32)
b = tf.keras.layers.Dense(32)

with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(a.variables)  # Since `a.build` has not been called at this point
                           # `a.variables` will return an empty list and the
                           # tape will not be watching anything. a.variables = []
  result = b(a(inputs))

tape.gradient(result, a.variables)  # The result of this computation will be
                                      # a list of `None`s since a's variables
                                      # are not being watched.

# **Tensorflow Graph concept: GradientTape**

In [ ]:
"""
@propety:
En Python, @property est un décorateur utilisé pour définir une méthode au sein d'une classe qui agit comme un attribut en lecture seule.
Lorsqu'une classe hérite d'une autre classe, la notation @property sur une méthode permet à la classe dérivée d'accéder à cette méthode comme
un attribut en lecture seule sans avoir besoin d'appeler explicitement la méthode. Cela permet d'encapsuler la logique de calcul d'un attribut au
sein de la classe et de fournir un accès plus propre à cet attribut pour les classes dérivées.

@tf.function est un décorateur, qui permet de transformer une fonction en un graphe de calcul tensorflow.
Il permet dans certains contextes d'accélérer le calcul. IL est conseillé si la fonction contient
des opérations tensorflow.
"""

class BaseClass:
    def __init__(self, value):
        self._value = value

    @property
    def value(self):
        return self._value

class DerivedClass(BaseClass):
    def __init__(self, value, extra):
        super().__init__(value)
        self.extra = extra

# Utilisation
obj = DerivedClass(42, "extra_data")
print(obj.value)  # Accès à l'attribut "value" comme s'il s'agissait d'un attribut direct


"""
Dans cet exemple, la classe BaseClass définit un attribut en lecture seule value en utilisant @property.
La classe dérivée DerivedClass peut y accéder comme si c'était un attribut direct,
même si la logique pour le calcul de value est encapsulée dans BaseClass.
"""

"""
Avec metrics, si on fait self.metrics on aura une liste de metriques
"""

In [ ]:
# *args
"""
En Python, *args est une syntaxe utilisée pour permettre à une fonction de recevoir un nombre variable d'arguments positionnels. Lorsque vous
utilisez *args comme paramètre dans la définition d'une fonction, cela signifie que la fonction peut accepter
un nombre arbitraire d'arguments positionnels, et ces arguments seront regroupés dans un tuple.
"""
def fonction_avec_args(arg1, *args):
    print("arg1:", arg1)
    print("args:", args)

# Utilisation de la fonction
fonction_avec_args(1, 2, 3, 4, 5)

"""
Dans cet exemple, arg1 est un argument positionnel classique, tandis que *args est utilisé pour collecter tous les arguments positionnels supplémentaires dans un tuple appelé args.
Lorsque la fonction est appelée avec plusieurs arguments après arg1, ils sont automatiquement regroupés dans le tuple args. Vous pouvez ensuite parcourir ce tuple pour traiter
les arguments supplémentaires comme vous le souhaitez.

L'utilisation de *args est courante lorsque vous devez définir une fonction qui peut prendre un nombre
variable d'arguments, par exemple, pour implémenter des fonctions génériques ou des wrappers autour d'autres fonctions.
"""

In [ ]:
# **kwargs
"""

En Python, **kwargs est une syntaxe similaire à *args, mais elle est utilisée pour permettre à une fonction de recevoir un nombre variable d'arguments mots-clés (ou paramètres nommés). Lorsque vous utilisez **kwargs
comme paramètre dans la définition d'une fonction, cela signifie que la fonction peut accepter un nombre arbitraire d'arguments mots-clés, et ces arguments seront regroupés dans un dictionnaire.
"""

def fonction_avec_kwargs(arg1, **kwargs):
    print("arg1:", arg1)
    print("kwargs:", kwargs)

# Utilisation de la fonction
fonction_avec_kwargs(1, a=2, b=3, c=4)

"""
Dans cet exemple, arg1 est un argument positionnel classique, tandis que **kwargs est utilisé pour collecter tous les
arguments mots-clés supplémentaires dans un dictionnaire appelé kwargs. Lorsque la fonction est appelée avec des arguments mots-clés supplémentaires (par exemple, a=2, b=3, c=4),
ces arguments sont regroupés dans le dictionnaire kwargs, où les noms des arguments sont les clés et leurs valeurs sont les valeurs correspondantes.

L'utilisation de **kwargs est courante lorsque vous devez définir une fonction qui peut accepter un nombre variable d'arguments mots-clés, ce qui peut être utile pour personnaliser
le comportement d'une fonction ou pour transmettre un grand nombre de paramètres de manière plus flexible.
"""

In [ ]:
"""
EXAMPLE : LOSS IS NOT GIVEN IN COMPILE
"""

class CustomModel(keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.mae_metric = keras.metrics.MeanAbsoluteError(name="mae")

    @property
    def metrics(self):
        # We list our `Metric` objects here so that `reset_states()` can be
        # called automatically at the start of each epoch
        # or at the start of `evaluate()`.
        # If you don't implement this property, you have to call
        # `reset_states()` yourself at the time of your choosing.
        return [self.loss_tracker, self.mae_metric] ## List all metrics

    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute our own loss
            loss = keras.losses.mean_squared_error(y, y_pred) ## if loss if given in comile, we can use by self.compute_loss

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars)) ## optimizer is given in self.optimizer

        # Compute our own metrics
        self.loss_tracker.update_state(loss)
        self.mae_metric.update_state(y, y_pred)
        return {"loss": self.loss_tracker.result(), "mae": self.mae_metric.result()} ## bar progress for user {m.name:m.result() for m in self.metrics}

    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False) ## Freeze weights
        loss = keras.losses.mean_squared_error(y, y_pred)
        self.loss_tracker.update_state(loss)
        self.mae_metric.update_state(y, y_pred)
        return {m.name:m.result() for m in self.metrics}

# Construct an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)

# We don't passs a loss or metrics here.
model.compile(optimizer="adam")

# Just use `fit` as usual -- you can use callbacks, etc.
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=5)

In [ ]:
"""
EXAMPLE : LOSS IS GIVEN IN COMPILE
"""

class CustomModel(keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.mae_metric = keras.metrics.MeanAbsoluteError(name="mae")

    @property
    def metrics(self):
        # We list our `Metric` objects here so that `reset_states()` can be
        # called automatically at the start of each epoch
        # or at the start of `evaluate()`.
        # If you don't implement this property, you have to call
        # `reset_states()` yourself at the time of your choosing.
        return [self.loss_tracker, self.mae_metric] ## List all metrics

    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute our own loss
            loss = self.compute_loss(y, y_pred) ### Changed line

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars)) ## optimizer is given in self.optimizer

        # Compute our own metrics
        self.loss_tracker.update_state(loss)
        self.mae_metric.update_state(y, y_pred)
        return {"loss": self.loss_tracker.result(), "mae": self.mae_metric.result()} ## bar progress for user {m.name:m.result() for m in self.metrics}

    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False) ## Freeze weights
        loss = self.compute_loss(y, y_pred) ### Changed line
        self.loss_tracker.update_state(loss)
        self.mae_metric.update_state(y, y_pred)
        return {m.name:m.result() for m in self.metrics}

# Construct an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)

# We don't passs a loss or metrics here.
model.compile(optimizer="adam", loss="mse")

# Just use `fit` as usual -- you can use callbacks, etc.
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=5)

In [ ]:
"""
EXAMPLE: PONDERATION OF CLASSES USING WEIGHTS
"""

class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        if len(data) == 3:
            x, y, sample_weight = data
        else:
            sample_weight = None
            x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value.
            # The loss function is configured in `compile()`.
            loss = self.compute_loss(
                y=y,
                y_pred=y_pred,
                sample_weight=sample_weight,
            )

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics.
        # Metrics are configured in `compile()`.
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(y, y_pred, sample_weight=sample_weight)

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# You can now use sample_weight argument
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
sw = np.random.random((1000, 1))
model.fit(x, y, sample_weight=sw, epochs=3)

# **Tensorflow Graph concept - GAN EXAMPLE: GradientTape**

In [ ]:
from tensorflow.keras import layers

# Create the discriminator
discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

# Create the generator
latent_dim = 128
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        # We want to generate 128 coefficients to reshape into a 7x7x128 map
        layers.Dense(7 * 7 * 128),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, 128)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_loss_tracker = keras.metrics.Mean(name="d_loss")
        self.g_loss_tracker = keras.metrics.Mean(name="g_loss")

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        if isinstance(real_images, tuple):
            real_images = real_images[0]
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape: ##  We can either use with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics and return their value.
        self.d_loss_tracker.update_state(d_loss)
        self.g_loss_tracker.update_state(g_loss)
        return {
            "d_loss": self.d_loss_tracker.result(),
            "g_loss": self.g_loss_tracker.result(),
        }

In [ ]:
# Prepare the dataset. We use both the training & test MNIST digits.
batch_size = 64
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_digits = all_digits.astype("float32") / 255.0
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
dataset = tf.data.Dataset.from_tensor_slices(all_digits)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

# To limit the execution time, we only train on 100 batches. You can train on
# the entire dataset. You will need about 20 epochs to get nice results.
gan.fit(dataset.take(100), epochs=1)

# **Tensorflow Graph concept - Image and Text Similarity: GradientTape**

In [ ]:
"""
EXAMPLE: In this example, the loss is computing on the class
         the optimizer is given in compile, we use self.optimizer to use it.
         https://keras.io/examples/vision/nl_image_search/
"""
class DualEncoder(keras.Model):
    def __init__(self, text_encoder, image_encoder, temperature=1.0, **kwargs):
        super().__init__(**kwargs)
        self.text_encoder = text_encoder
        self.image_encoder = image_encoder
        self.temperature = temperature
        self.loss_tracker = keras.metrics.Mean(name="loss")

    @property
    def metrics(self):
        return [self.loss_tracker]

    def call(self, features, training=False):
        # Place each encoder on a separate GPU (if available).
        # TF will fallback on available devices if there are fewer than 2 GPUs.
        with tf.device("/gpu:0"):
            # Get the embeddings for the captions.
            caption_embeddings = text_encoder(features["caption"], training=training)
        with tf.device("/gpu:1"):
            # Get the embeddings for the images.
            image_embeddings = vision_encoder(features["image"], training=training)
        return caption_embeddings, image_embeddings

    def compute_loss(self, caption_embeddings, image_embeddings):
        # logits[i][j] is the dot_similarity(caption_i, image_j).
        logits = (
            tf.matmul(caption_embeddings, image_embeddings, transpose_b=True)
            / self.temperature
        )
        # images_similarity[i][j] is the dot_similarity(image_i, image_j).
        images_similarity = tf.matmul(
            image_embeddings, image_embeddings, transpose_b=True
        )
        # captions_similarity[i][j] is the dot_similarity(caption_i, caption_j).
        captions_similarity = tf.matmul(
            caption_embeddings, caption_embeddings, transpose_b=True
        )
        # targets[i][j] = avarage dot_similarity(caption_i, caption_j) and dot_similarity(image_i, image_j).
        targets = keras.activations.softmax(
            (captions_similarity + images_similarity) / (2 * self.temperature)
        )
        # Compute the loss for the captions using crossentropy
        captions_loss = keras.losses.categorical_crossentropy(
            y_true=targets, y_pred=logits, from_logits=True
        )
        # Compute the loss for the images using crossentropy
        images_loss = keras.losses.categorical_crossentropy(
            y_true=tf.transpose(targets), y_pred=tf.transpose(logits), from_logits=True
        )
        # Return the mean of the loss over the batch.
        return (captions_loss + images_loss) / 2

    def train_step(self, features):
        with tf.GradientTape() as tape:
            # Forward pass
            caption_embeddings, image_embeddings = self(features, training=True)
            loss = self.compute_loss(caption_embeddings, image_embeddings)
        # Backward pass
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # Monitor loss
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, features):
        caption_embeddings, image_embeddings = self(features, training=False)
        loss = self.compute_loss(caption_embeddings, image_embeddings)
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

In [ ]:
num_epochs = 5  # In practice, train for at least 30 epochs
batch_size = 256

vision_encoder = create_vision_encoder(
    num_projection_layers=1, projection_dims=256, dropout_rate=0.1
)
text_encoder = create_text_encoder(
    num_projection_layers=1, projection_dims=256, dropout_rate=0.1
)
dual_encoder = DualEncoder(text_encoder, vision_encoder, temperature=0.05)
dual_encoder.compile(
    optimizer=tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.001)
)

# **Tensorflow Graph concept - Relevant method for GAN - @tf.function: GradientTape**

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0, beta_2=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0, beta_2=0.9)

# save checkpoints
checkpoint_dir = "training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                discriminator_optimizer=discriminator_optimizer,
                                generator=generator,
                                discriminator=discriminator)

# Constant
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16
seed = tf.random.normal((num_examples_to_generate, noise_dim))

# @tf.function enables to use compile with train_step
@tf.function
def train_step(images):
    noise = tf.random.normal((BATCH_SIZE, noise_dim))
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)


    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

# Train function
def train(datasets, epochs):
    for epoch in range(epochs):
        start = time.time()

        for img_batch in datasets:
            train_step(img_batch)

        # Produce images for the GIF
        display.clear_output(wait=True)
        generate_and_save_images(generator,
                                epoch+1,
                                seed)

        # Save the model every 15 epochs
        if (epoch+1)%15 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)


        print("Time for epoch {} is {} sec".format(epoch+1, time.time()-start))

    # Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                                epoch,
                                seed)

# Predicion
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(8,8))

    for i in range(predictions.shape[0]):
        plt.subplot(4,4,i+1)
        plt.imshow(predictions[i, :, :, :]*127.5 + 127.5)
        plt.axis('off')

    plt.savefig('generated_image/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

# Train
train(train_dataset, EPOCHS)

# Restore
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

# Plot
def display_image(epoch):
    return PIL.Image.open("generated_image/image_at_epoch_{:04d}.png".format(epoch))
display_image(EPOCHs)

In [ ]:
"""
We can use an other approach
"""
# @tf.function enables to use compile with train_step
@tf.function
def train_step(images):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

        # Train discriminator
        noise = tf.random.normal((BATCH_SIZE, noise_dim))
        generated_images = generator.predict(noise)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        disc_loss = discriminator_loss(real_output, fake_output)

        # Train generator
        noise = tf.random.normal((BATCH_SIZE, noise_dim))
        generated_images = generator(noise, training=True)
        fake_output = discriminator(generated_images, training=False)
        gen_loss = generator_loss(fake_output)



    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))



# **Method using train_on_batch**

In [ ]:
"""
This method use an other approach for training : train_on_batch
https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-a-cifar-10-small-object-photographs-from-scratch/
"""

# Define the standalone discriminator model
def define_discriminator(in_shape=(32, 32, 3)):
    in_image = Input(shape=in_shape)
    # Normal
    x = Conv2D(64, (3, 3), padding='same')(in_image)
    x = LeakyReLU(alpha=0.2)(x)
    # Downsample
    x = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    # Downsample
    x = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    # Downsample
    x = Conv2D(256, (3, 3), strides=(2, 2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    # Classifier
    x = Flatten()(x)
    x = Dropout(0.4)(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(in_image, x)

    # Compile model
    opt = Adam(learning_rate=0.0002, beta_1=0, beta_2=0.9)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

# Define the standalone generator model
def define_generator(latent_dim):
    in_lat = Input(shape=(latent_dim,))
    # Foundation for 4x4 image
    n_nodes = 256 * 4 * 4
    x = Dense(n_nodes)(in_lat)
    x = LeakyReLU(alpha=0.2)(x)
    x = Reshape((4, 4, 256))(x)

    # Upsample to 8x8
    x = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x, training=True)
    x = LeakyReLU(alpha=0.2)(x)

    # Upsample to 16x16
    x = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x, training=True)
    x = LeakyReLU(alpha=0.2)(x)

    # Upsample to 32x32
    x = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x, training=True)
    x = LeakyReLU(alpha=0.2)(x)

    # Output layer
    x = Conv2D(3, (3, 3), activation='tanh', padding='same')(x)

    model = Model(in_lat, x)

    return model

In [ ]:
"""
Define the gan model that combine discriminator that is frozen and generator that is not frozen
NB: Freeze discriminator
    Not Freeze generator
"""
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0, beta_2=0.9)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model


In [ ]:
# load and prepare cifar10 training images
def load_real_samples():
    # load cifar10 dataset
    (trainX, _), (_, _) = load_data()
    # convert from unsigned ints to floats
    X = trainX.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X


# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y


# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input


# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input, verbose=0)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

In [ ]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
     # scale from [-1,1] to [0,1]
     examples = (examples + 1) / 2.0
     # plot images
     for i in range(n * n):
         # define subplot
         pyplot.subplot(n, n, 1 + i)
         # turn off axis
         pyplot.axis('off')
         # plot raw pixel data
         pyplot.imshow(examples[i])
     # save plot to file
     filename = 'generated_plot_e%03d.png' % (epoch+1)
     pyplot.savefig(filename)
     pyplot.show()

In [ ]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
     # prepare real samples
     X_real, y_real = generate_real_samples(dataset, n_samples)
     # evaluate discriminator on real examples
     _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
     # prepare fake examples
     x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
     # evaluate discriminator on fake examples
     _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
     # summarize discriminator performance
     print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
     # save plot
     save_plot(x_fake, epoch)
     # save the generator model tile file
     filename = 'generator_model_%03d.h5' % (epoch+1)
     g_model.save(filename)



In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
     bat_per_epo = int(dataset.shape[0] / n_batch)
     half_batch = int(n_batch / 2)
     # manually enumerate epochs
     for i in range(n_epochs):
     # enumerate batches over the training set
         for j in range(bat_per_epo):
            """
              Train the discriminator
              NB: The same discriminator and generator is used in all training
            """
             # get randomly selected 'real' samples
             X_real, y_real = generate_real_samples(dataset, half_batch)
             # update discriminator model weights
             d_loss1, _ = d_model.train_on_batch(X_real, y_real)
             # generate 'fake' examples
             X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
             # update discriminator model weights
             d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)


            """
              Train the generator
            """
             # prepare points in latent space as input for the generator
             X_gan = generate_latent_points(latent_dim, n_batch)
             # create inverted labels for the fake samples
             y_gan = ones((n_batch, 1))
             # update the generator via the discriminator's error
             g_loss = gan_model.train_on_batch(X_gan, y_gan)
             # summarize loss on this batch
         print('epoch %d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
         (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
         # evaluate the model performance, sometimes
         if (i+1) % 10 == 0:
             summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [ ]:
"""
NB: d_model et g_model sont utilisés pour créer gan_model
    Il faut noter que d_model.train_on_batch va mettre à jour les poids de d_model dans gan_model (c'est le même d_model)
    aussi toutes les modifications de g_model vont répercuter sur le g_model dans gan_model donc c'est le même
    """

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

# **Machine Translation**

In [ ]:
"""
Dans cet exemple, on pose l'hypothèse que les données sont déjà représentées sous forme de séquence c'est à dire qu'on n'a
pas besoin de la couche d'Embedding. Dans le cas courant, où les données sont représentées sous forme de vecteur, il faut nécessairement un embedding
pour avoir chaque mot représenté en vecteur.
https://www.kaggle.com/code/akshat0007/machine-translation-english-to-french-rnn-lstm
"""

In [ ]:
# Encoder
input_encoder = Input(shape=(None, features))
encoder = LSTM(units=latent_dim, return_states=True)
output_encoder, state_h, state_c = encoder(input_encoder)

# Decoder
input_decoder = Input(shape=(None, features))
decoder = LSTM(units=latent_dim, return_states=True, return_sequences=True)
output_lstm, _, _ = decoder(input_decoder, initial_state=[state_h, state_c])
dense_layer = Dense(units=vocab_size, activation="softmax")
dense_output = dense_layer(output_lstm)

# Model
model = Model([input_encoder, input_encoder], dense_output)

# Compile model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]) # one hot in label

# Train
model.fit([encoder_data, decoder_data], y_decoder, epochs=100, batch_size=32)

In [ ]:
"""
After training, we build an encoder and decoder depending on the training model for prediction
"""

# Build Encoder
encoder = Model(input_encoder, [output_encoder, state_h, state_c])

# Build Decoder
input_decoder = Input(shape=(1, features))
input_h = Input(shape=(latent_dim, ))
input_c = Input(shape=(latent_dim, ))
output_lstm, state_h state_c = decoder(input_decoder, initial_state=[input_h, input_c])
output_decoder = dense_layer(output_lstm)

decoder = Model([input_decoder, input_h, input_c], [output_decoder, state_h, state_c])

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

# **BahdanauAttention in LSTM using Hidden state as query**

In [ ]:
"""
ChatBot Introduction Colab: https://www.kaggle.com/code/alincijov/dialog-chatbot-using-bahdanau-attention
"""

In [ ]:
# Split the dataset for train and test
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

BUFFER_SIZE = len(input_tensor_train)  # Define the buffer size, typically the number of training examples.
BATCH_SIZE = 64  # Define the batch size for training data.
steps_per_epoch = len(input_tensor_train) // BATCH_SIZE  # Calculate the number of steps per training epoch.
embedding_dim = 256  # Define the dimension of word embeddings.
units = 1024  # Define the number of units or neurons in a recurrent neural network (RNN) layer.
vocab_inp_size = len(inp_lang.word_index) + 1  # Calculate the size of the input vocabulary.
vocab_tar_size = len(targ_lang.word_index) + 1  # Calculate the size of the target vocabulary.

# Create a TensorFlow dataset from the input and target tensors, and shuffle it using the specified BUFFER_SIZE.
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)

# Batch the dataset into batches of BATCH_SIZE and drop any remaining examples that don't fit into a batch.
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

# Get an example input batch and an example target batch from the dataset.
example_input_batch, example_target_batch = next(iter(dataset))

# Print the shapes of the example input and target batches.
print("Example Input Batch Shape:", example_input_batch.shape)
print("Example Target Batch Shape:", example_target_batch.shape)

In [ ]:
"""
BahdanauAttention: prend en entrée le hidden_state de l'encoder considéré comme query, et la sortie de l'encoder considéré comme key et value
La sortie de la couche BahdanauAttention utilisée dans le décodeur et concaténer avec le mot en cours de traitement (son embedding)
"""

In [ ]:
# Define a custom class called "Encoder" that inherits from the tf.keras.Model class.
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()  # Call the constructor of the parent class.
        self.batch_sz = batch_sz  # Store the batch size as an instance variable.
        self.enc_units = enc_units  # Store the number of units in the GRU layer.

        # Create an embedding layer to convert input tokens into dense vectors.
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

        # Create a GRU (Gated Recurrent Unit) layer with specified parameters.
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    # Define the forward pass for the encoder.
    def call(self, x, hidden):
        x = self.embedding(x)  # Pass the input through the embedding layer. --------------------------------(None, sequence, embedding_dim)
        output, state = self.gru(x, initial_state=hidden)  # Pass through the GRU. --------------------------(None, sequence, enc_units), (None, enc_units)
        return output, state  # Return the output sequence and final hidden state. --------------------------(None, sequence, enc_units), (None, enc_units)

    # Initialize the hidden state (typically with zeros).
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
"""
Q = Decoder hidden state (None, decoder_unit)
    at the begining, Q = Encoder hidden state

V = Encoder output (None, sequence, feature)

Q = tf.expand_dims(Q, axis=1) # (None, 1, decoder_unit)
EncQ = Dense1(Q) # (None, units)
EncV = Dense2(V) # (None, sequence, units)
sum = tf.nn.tanh(EncQ+EncV) # activation (None, sequence, units)

score = Dense3(sum) # (None, sequence, 1)
attention = tf.nn.softmax(score, axis=1) # (None, sequence, 1) softmax based on token position

context_vector = attention*V # (None, sequence, feature)
context_vector = tf.nn.sum(context_vector, axis=1) # (None, feature)

context_vector sera utilisé dans le decoder et concaténer avec l'entrée (qui se fait token par token)
x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1) # (None, 1, feature+embedding_dim_x) x vient de l'embedding x est de type (None, 1) car on traite par token Embedding donne (None, 1, embedding_dim_x)
"""

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        """
          query = hidden state dec = (None, latent_dim)
          values = outputs of enc = (None, sequence, latent_dim)
        """
        # (None, 1, latent_dim)
        query_with_time_axis = tf.expand_dims(query, 1)

        # (None, sequence, 1)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # (batch_size, sequence, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # (None, sequence, latent_dim)
        context_vector = attention_weights * values

        # (None, latent_dim)
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0)) # return False if 0 and True else (0 is the pad token)
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask # reduce to 0 if correspond to pad token (False)

    return tf.reduce_mean(loss_)

In [ ]:
import tensorflow as tf
loss = 1
print(tf.reduce_mean([0, 0.2,0.3]))
loss+=tf.reduce_mean([0, 0.2,0.3])
print(loss)

tf.Tensor(0.16666667, shape=(), dtype=float32)
tf.Tensor(1.1666666, shape=(), dtype=float32)


In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        # (BATCH_SIZE, 1)
        dec_input = tf.expand_dims([targ_lang.word_index['<sos>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [ ]:
EPOCHS = 40

# Training taking batch per batch
for epoch in range(1, EPOCHS + 1):
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

    if(epoch % 4 == 0):
        print('Epoch:{:3d} Loss:{:.4f}'.format(epoch,
                                          total_loss / steps_per_epoch))

# **tf.data.Dataset**

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_generator
# + load image: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

import tensorflow as tf

In [ ]:
import numpy as np
l = [1,2,9,8,4]
np.random.shuffle(l)

In [ ]:
l

[2, 9, 1, 8, 4]

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence

class CustomDataGenerator(Sequence):
    def __init__(self, x_data, y_data, batch_size, shuffle=True):
        self.x_data = x_data
        self.y_data = y_data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.x_data))

        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.ceil(len(self.x_data) / self.batch_size)) # np.ceil : partie entière inférieure -->

    def __getitem__(self, index):
        start = index * self.batch_size
        end = (index + 1) * self.batch_size

        batch_x = self.x_data[self.indexes[start:end]]
        batch_y = self.y_data[self.indexes[start:end]]
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

# Example usage:
# Create some example data
x_data = np.random.rand(100, 32, 32, 3)
y_data = np.random.randint(0, 2, size=(100,))

# Create an instance of the custom data generator
batch_size = 32
data_generator = CustomDataGenerator(x_data, y_data, batch_size)

# Iterate through the data generator for training
for epoch in range(epochs):
    for batch_x, batch_y in data_generator:
        # Train your model on batch_x and batch_y


**Source dataset**

In [ ]:
# Data don't fit in memory
data = [1,2,3]
dataset = tf.data.Dataset.from_tensor_slices(data)
for element in dataset:
    print(element)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)


In [ ]:
# Read lines from Files
files = ["text1", "text2"]
dataset = tf.data.TextLineDataset(files)
for element in dataset:
    print(element)

tf.Tensor(b'Le senegal', shape=(), dtype=string)
tf.Tensor(b'La gambie', shape=(), dtype=string)
tf.Tensor(b'Le Maroc', shape=(), dtype=string)
tf.Tensor(b'La Tunisie', shape=(), dtype=string)


In [ ]:
# Read name of files giving extension
path = '*.txt'
dataset = tf.data.Dataset.list_files(path)
for element in dataset:
    print(element)

tf.Tensor(b'./text1.txt', shape=(), dtype=string)
tf.Tensor(b'./text2.txt', shape=(), dtype=string)


**Transformation**

In [ ]:
# map
map(
    map_func,
    num_parallel_calls=None,  # representing the number elements to process asynchronously in parallel. If not specified, elements will be processed sequentially. If the value tf.data.AUTOTUNE is used, then the number of parallel calls is set dynamically based on available CPU.
    deterministic=None,
    name=None
)

In [ ]:
# 1. map
dataset = tf.data.Dataset.from_tensor_slices([1,2,3])
dataset = dataset.map(lambda x: x**2)
list(dataset.as_numpy_iterator())

[1, 4, 9]

In [ ]:
# 2. map
dataset = tf.data.Dataset.from_tensor_slices(([1,2,3], [0,1,1]))
dataset = dataset.map(lambda x1, x2: (x1**2, x2))
list(dataset.as_numpy_iterator())

[(1, 0), (4, 1), (9, 1)]

In [ ]:
# filter
dataset = tf.data.Dataset.range(100)
dataset = dataset.filter(lambda x: x < 5)
list(dataset.as_numpy_iterator())

[0, 1, 2, 3, 4]

In [ ]:
# apply
dataset = tf.data.Dataset.range(100)

def my_filter(ds):
    return ds.filter(lambda x: x < 5)

dataset = dataset.apply(my_filter)
list(dataset.as_numpy_iterator())

[0, 1, 2, 3, 4]

In [ ]:
# as_numpy_iterator
dataset = tf.data.Dataset.from_tensor_slices({'a': ([1, 2], [3, 4]),
                                              'b': [5, 6]})
list(dataset.as_numpy_iterator()) == [{'a': (1, 3), 'b': 5},
                                      {'a': (2, 4), 'b': 6}]


True

In [ ]:
# batch
batch(
    batch_size,
    drop_remainder=False, # representing whether the last batch should be dropped in the case it has fewer than batch_size elements; the default behavior is not to drop the smaller batch.
    num_parallel_calls=None, # number of batches to compute asynchronously in parallel tf.data.AUTOTUNE
    deterministic=None,
    name=None
)

In [ ]:
# batch
dataset = tf.data.Dataset.range(8)
dataset = dataset.batch(3)
list(dataset.as_numpy_iterator())

[array([0, 1, 2]), array([3, 4, 5]), array([6, 7])]

In [ ]:
dataset = tf.data.Dataset.range(8)
dataset = dataset.batch(3, drop_remainder=True)
list(dataset.as_numpy_iterator())

[array([0, 1, 2]), array([3, 4, 5])]

In [ ]:
# cardinality
dataset = tf.data.Dataset.range(42)
print(dataset.cardinality().numpy())

42


In [ ]:
# flat_map
dataset = tf.data.Dataset.from_tensor_slices(
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]])
dataset = dataset.flat_map(tf.data.Dataset.from_tensor_slices)
list(dataset.as_numpy_iterator())

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
# from_tensors: Creates a Dataset with a single element, comprising the given tensors.
dataset = tf.data.Dataset.from_tensors([1, 2, 3])
for element in dataset:
    print(element)

tf.Tensor([1 2 3], shape=(3,), dtype=int32)


In [ ]:
# from_tensors: Creates a Dataset with a single element, comprising the given tensors.
dataset = tf.data.Dataset.from_tensors([1, 2, 3])
list(dataset.as_numpy_iterator())

[array([1, 2, 3], dtype=int32)]

**Make dataset**

In [ ]:
data = tf.data.Dataset.range(10)
dataset = (
    data
    .map(lambda x: x**2, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(2)
)

for element in dataset:
  print(element)

tf.Tensor([0 1], shape=(2,), dtype=int64)
tf.Tensor([4 9], shape=(2,), dtype=int64)
tf.Tensor([16 25], shape=(2,), dtype=int64)
tf.Tensor([36 49], shape=(2,), dtype=int64)
tf.Tensor([64 81], shape=(2,), dtype=int64)


In [ ]:
# Slicing a 1D tensor produces scalar tensor elements.
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
list(dataset.as_numpy_iterator())

[1, 2, 3]

In [ ]:
# Slicing a 2D tensor produces 1D tensor elements.
dataset = tf.data.Dataset.from_tensor_slices([[1, 2], [3, 4]])
list(dataset.as_numpy_iterator())

[array([1, 2], dtype=int32), array([3, 4], dtype=int32)]

In [ ]:
# Slicing a tuple of 1D tensors produces tuple elements containing WAY FOR DATASET PROCESSING
# scalar tensors.
dataset = tf.data.Dataset.from_tensor_slices(([1, 2], [3, 4], [5, 6]))
list(dataset.as_numpy_iterator())

[(1, 3, 5), (2, 4, 6)]

In [ ]:
# Dictionary structure is also preserved.
dataset = tf.data.Dataset.from_tensor_slices({"a": [1, 2], "b": [3, 4]})
list(dataset.as_numpy_iterator())


[{'a': 1, 'b': 3}, {'a': 2, 'b': 4}]

In [ ]:
# Two tensors can be combined into one Dataset object.
features = tf.constant([[1, 3], [2, 1], [3, 3]]) # ==> 3x2 tensor
labels = tf.constant(['A', 'B', 'A']) # ==> 3x1 tensor
dataset = tf.data.Dataset.from_tensor_slices((features, labels))
list(dataset.as_numpy_iterator())

[(array([1, 3], dtype=int32), b'A'),
 (array([2, 1], dtype=int32), b'B'),
 (array([3, 3], dtype=int32), b'A')]

In [ ]:
# Two tensors can be combined into one Dataset object.
features = tf.constant([[1, 3], [2, 1], [3, 3]]) # ==> 3x2 tensor
labels = tf.constant(['A', 'B', 'A']) # ==> 3x1 tensor
dataset = tf.data.Dataset.from_tensor_slices((features, labels))
# Both the features and the labels tensors can be converted
# to a Dataset object separately and combined after.
features_dataset = tf.data.Dataset.from_tensor_slices(features)
labels_dataset = tf.data.Dataset.from_tensor_slices(labels)
dataset = tf.data.Dataset.zip((features_dataset, labels_dataset))
list(dataset.as_numpy_iterator())

[(array([1, 3], dtype=int32), b'A'),
 (array([2, 1], dtype=int32), b'B'),
 (array([3, 3], dtype=int32), b'A')]

In [ ]:
# A batched feature and label set can be converted to a Dataset
# in similar fashion.
batched_features = tf.constant([[[1, 3], [2, 3]],
                                [[2, 1], [1, 2]],
                                [[3, 3], [3, 2]]], shape=(3, 2, 2))
batched_labels = tf.constant([['A', 'A'],
                              ['B', 'B'],
                              ['A', 'B']], shape=(3, 2, 1))
dataset = tf.data.Dataset.from_tensor_slices((batched_features, batched_labels))
for element in dataset.as_numpy_iterator():
    print(element)
# show shape
print(dataset)

(array([[1, 3],
       [2, 3]], dtype=int32), array([[b'A'],
       [b'A']], dtype=object))
(array([[2, 1],
       [1, 2]], dtype=int32), array([[b'B'],
       [b'B']], dtype=object))
(array([[3, 3],
       [3, 2]], dtype=int32), array([[b'A'],
       [b'B']], dtype=object))
<_TensorSliceDataset element_spec=(TensorSpec(shape=(2, 2), dtype=tf.int32, name=None), TensorSpec(shape=(2, 1), dtype=tf.string, name=None))>


In [ ]:
# A batched feature and label set can be converted to a Dataset
# in similar fashion.
batched_features = tf.constant([[[1, 3], [2, 3]],
                                [[2, 1], [1, 2]],
                                [[3, 3], [3, 2]]], shape=(3, 2, 2))
batched_labels = tf.constant([['A', 'A'],
                              ['B', 'B'],
                              ['A', 'B']], shape=(3, 2)) # change shape
dataset = tf.data.Dataset.from_tensor_slices((batched_features, batched_labels))
for element in dataset.as_numpy_iterator():
    print(element)
# show shape
print(dataset)

(array([[1, 3],
       [2, 3]], dtype=int32), array([b'A', b'A'], dtype=object))
(array([[2, 1],
       [1, 2]], dtype=int32), array([b'B', b'B'], dtype=object))
(array([[3, 3],
       [3, 2]], dtype=int32), array([b'A', b'B'], dtype=object))
<_TensorSliceDataset element_spec=(TensorSpec(shape=(2, 2), dtype=tf.int32, name=None), TensorSpec(shape=(2,), dtype=tf.string, name=None))>


**prefetch**

Creates a Dataset that prefetches elements from this dataset.

Most dataset input pipelines should end with a call to prefetch. This allows later elements to be prepared while the current element is being processed. This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.

Note: Like other Dataset methods, prefetch operates on the elements of the input dataset. It has no concept of examples vs. batches. examples.prefetch(2) will prefetch two elements (2 examples), while examples.batch(20).prefetch(2) will prefetch 2 elements (2 batches, of 20 examples each).

In [ ]:
prefetch(
    buffer_size, # 	A tf.int64 scalar tf.Tensor, representing the maximum number of elements that will be buffered when prefetching. If the value tf.data.AUTOTUNE is used, then the buffer size is dynamically tuned.
    name=None
)

In [ ]:
dataset = tf.data.Dataset.range(3)
dataset = dataset.prefetch(2)
list(dataset.as_numpy_iterator())

[0, 1, 2]

**shuffle**

In [ ]:
dataset = tf.data.Dataset.range(3)
dataset = dataset.shuffle(3, reshuffle_each_iteration=False)
dataset = dataset.repeat(2)
list(dataset.as_numpy_iterator())

[0, 2, 1, 0, 2, 1]

In [ ]:
dataset = tf.data.Dataset.range(3)
dataset = dataset.shuffle(3, reshuffle_each_iteration=True)
dataset = dataset.repeat(2)
list(dataset.as_numpy_iterator())

[2, 0, 1, 0, 2, 1]

**take**

In [ ]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.take(3)
list(dataset.as_numpy_iterator())


[0, 1, 2]

**take_while**

In [ ]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.take_while(lambda x: x < 5)
list(dataset.as_numpy_iterator())

[0, 1, 2, 3, 4]

**unbatch**

Splits elements of a dataset into multiple elements.

For example, if elements of the dataset are shaped [B, a0, a1, ...], where B may vary for each input element, then for each element in the dataset, the unbatched dataset will contain B consecutive elements of shape [a0, a1, ...].

In [ ]:
elements = [ [1, 2, 3], [1, 2], [1, 2, 3, 4] ]
dataset = tf.data.Dataset.from_generator(lambda: elements, tf.int64)
dataset = dataset.unbatch()
list(dataset.as_numpy_iterator())

[1, 2, 3, 1, 2, 1, 2, 3, 4]

**unique**

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1, 37, 2, 37, 2, 1])
dataset = dataset.unique()
sorted(list(dataset.as_numpy_iterator()))

[1, 2, 37]

**window**

Returns a dataset of "windows".

Each "window" is a dataset that contains a subset of elements of the input dataset. These are finite datasets of size size (or possibly fewer if there are not enough input elements to fill the window and drop_remainder evaluates to False).



In [ ]:
dataset = tf.data.Dataset.range(7).window(3,
                                          drop_remainder=True)
for window in dataset:
  print(list(window.as_numpy_iterator()))

[0, 1, 2]
[3, 4, 5]


The shift argument determines the number of input elements to shift between the start of each window. If windows and elements are both numbered starting at 0, the first element in window k will be element k * shift of the input dataset. In particular, the first element of the first window will always be the first element of the input dataset.

In [ ]:
dataset = tf.data.Dataset.range(7).window(3, shift=1,
                                          drop_remainder=True)
for window in dataset:
    print(list(window.as_numpy_iterator()))

[0, 1, 2]
[1, 2, 3]
[2, 3, 4]
[3, 4, 5]
[4, 5, 6]


The stride argument determines the stride between input elements within a window.

In [ ]:
dataset = tf.data.Dataset.range(7).window(3, shift=1, stride=2,
                                          drop_remainder=True)
for window in dataset:
  print(list(window.as_numpy_iterator()))

[0, 2, 4]
[1, 3, 5]
[2, 4, 6]


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(([1, 2, 3, 4, 5],
                                              [6, 7, 8, 9, 10]))
dataset = dataset.window(2)
print(dataset)

<_WindowDataset element_spec=(DatasetSpec(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorShape([])), DatasetSpec(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorShape([])))>


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(([1, 2, 3, 4, 5],
                                              [6, 7, 8, 9, 10]))
dataset = dataset.window(2)

for w in dataset: # w is a tuple
  print("#############")
  print(w[0].batch(2))
  print(list(w[0].as_numpy_iterator()))
  print(list(w[1].as_numpy_iterator()))

#############
<_BatchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>
[1, 2]
[6, 7]
#############
<_BatchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>
[3, 4]
[8, 9]
#############
<_BatchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>
[5]
[10]


In [ ]:
dataset = tf.data.Dataset.range(7).window(3, shift=1,
                                          drop_remainder=True)
batched = dataset.flat_map(lambda x:x.batch(3))
for batch in batched:
  print(batch)
  print(batch.numpy())

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
[0 1 2]
tf.Tensor([1 2 3], shape=(3,), dtype=int64)
[1 2 3]
tf.Tensor([2 3 4], shape=(3,), dtype=int64)
[2 3 4]
tf.Tensor([3 4 5], shape=(3,), dtype=int64)
[3 4 5]
tf.Tensor([4 5 6], shape=(3,), dtype=int64)
[4 5 6]


**zip**

In [ ]:
a = tf.data.Dataset.range(1, 4)  # ==> [ 1, 2, 3 ]
b = tf.data.Dataset.range(4, 7)  # ==> [ 4, 5, 6 ]
ds = tf.data.Dataset.zip(a, b)
list(ds.as_numpy_iterator())

[(1, 4), (2, 5), (3, 6)]

In [ ]:
ds = tf.data.Dataset.zip(b, a)
list(ds.as_numpy_iterator())

[(4, 1), (5, 2), (6, 3)]

In [ ]:
# The `datasets` argument may contain an arbitrary number of datasets.
c = tf.data.Dataset.range(7, 13).batch(2)  # ==> [ [7, 8],
                                           #       [9, 10],
                                           #       [11, 12] ]
ds = tf.data.Dataset.zip(a, b, c)
for element in ds.as_numpy_iterator():
  print(element)

(1, 4, array([7, 8]))
(2, 5, array([ 9, 10]))
(3, 6, array([11, 12]))


In [ ]:
# The number of elements in the resulting dataset is the same as
# the size of the smallest dataset in `datasets`.
d = tf.data.Dataset.range(13, 15)  # ==> [ 13, 14 ]
ds = tf.data.Dataset.zip(a, d)
list(ds.as_numpy_iterator())

[(1, 13), (2, 14)]

In [ ]:
a = tf.data.Dataset.from_tensor_slices(([1,2,3], [5,5,7]))
for i in a:
  print(i)

(<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=5>)
(<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=5>)
(<tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=7>)


# **Image - Text - Sequence**

* **Module**

* **Class**

* **Function**

In [ ]:
"""
Sur Tensorflow, on a des modules, des classes et des fonctions
tf.keras.preprocessing: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing [Module: image, text, sequence]
tf.keras.utils: https://www.tensorflow.org/api_docs/python/tf/keras/utils: exemple :
tf.keras.utils.to_categorical(
    y, num_classes=None, dtype='float32'
)
"""

In [ ]:
# Fine-tune model application
https://keras.io/api/applications/

# **Callbacks**

In [ ]:
"""
callbacks permet de controler le train
tf.keras.callbacks.Callback
https://www.tensorflow.org/api_docs/python/tf/keras/callbacks [Callback, ModelCheckpoint]
"""

In [ ]:
import tensorflow as tf
from tensorflow import keras
"""
self c'est le modèle
"""
class MyCustomCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Starting epoch {epoch}")

    def on_epoch_end(self, epoch, logs=None):
        print(f"Finished epoch {epoch}. Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}")

    def on_train_batch_begin(self, batch, logs=None):
        print(f"Training batch {batch}...")

    def on_train_batch_end(self, batch, logs=None):
        print(f"Finished training batch {batch}. Loss: {logs['loss']:.4f}")

    def on_test_begin(self, logs=None):
        print("Starting validation...")

    def on_test_end(self, logs=None):
        print("Finished validation. Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}")

    def on_predict_begin(self, logs=None):
        print("Starting prediction...")

    def on_predict_end(self, logs=None):
        print("Finished prediction.")

# Créez un modèle Keras simple à des fins d'exemple
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(784,)),
    keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Générez des données factices pour l'exemple
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

# Entraînez le modèle avec l'utilisation de votre callback personnalisé
history = model.fit(x_train, y_train, epochs=3, callbacks=[MyCustomCallback()])

# Effectuez une évaluation du modèle
model.evaluate(x_test, y_test, callbacks=[MyCustomCallback()])

# Effectuez des prédictions
model.predict(x_test[:5], callbacks=[MyCustomCallback()])


In [ ]:
checkpoint_dir = 'drive/MyDrive/autoACN_densenet40_cifar100'
# Créer un callback pour enregistrer les checkpoints tous les 50 époques
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_weights_{epoch:03d}.h5'),
    save_weights_only=True,
    save_best_only=False,
    save_freq=SAVE_PERIOD*STEPS_PER_EPOCH # Sauvegarder tous les 50 époques
)
# Train the DenseNet-40 model
history = model_densenet40.fit(train_gen,
                     steps_per_epoch=x_train.shape[0] // batch_size, epochs=epochs,
                     validation_data=(x_val, y_val), callbacks=[keras.callbacks.LearningRateScheduler(lr_schedule), checkpoint_callback])


In [ ]:
print('Restored model, accuracy: {:5.2f}%'.format(100 * 2.08))

Restored model, accuracy: 208.00%


In [ ]:
# Exemple plus avancé
class GradientStabilityCallback(keras.callbacks.Callback):
    def __init__(self, source_ds, target_ds):
        self.source_ds = source_ds
        self.target_ds = target_ds
        self.gradient_variances_source = []
        self.gradient_variances_target = []

    def on_epoch_end(self, epoch, logs=None):
        trainable_weights = self.model.model.trainable_weights
        gradients_source = []
        gradients_target = []

        # Choisissez un lot de données à partir de l'ensemble source
        source_batch = next(iter(self.source_ds))
        target_batch = next(iter(self.target_ds))

        with tf.GradientTape(persistent=True) as tape_source,  tf.GradientTape(persistent=True) as tape_target:
            # Passe avant avec les données source
            source_input = source_batch[0][0] # (x, y, context), (x, y, context)
            target_input = target_batch[0][0]

            source_logits = self.model.model(source_input, training=True)
            target_logits = self.model.model(target_input, training=True)


        # trainable_weights: liste de tf.Variables, avec chaque tf.Variables les poids d'une couche (une matrice)
        for weight in trainable_weights:
            # Vérifiez si le poids est connecté au calcul de source_logits
            gradients_source.append(tape_source.gradient(source_logits, weight)) # weight est de type tf.Variables et contient les poids d'une couche donnée
            gradients_target.append(tape_target.gradient(target_logits, weight))


        gradient_variance_source = [np.var(grad.numpy()) for grad in gradients_source]
        gradient_variance_target = [np.var(grad.numpy()) for grad in gradients_target]

        # Enregistrez ou stockez éventuellement gradient_variance selon vos besoins

        self.gradient_variances_source.append(gradient_variance_source)
        self.gradient_variances_target.append(gradient_variance_target)
        #print(f"Époque {epoch + 1}, Variance des gradients : {gradient_variance}")

# Create an instance of the GradientStabilityCallback with your source and target datasets
gradient_stability_callback = GradientStabilityCallback(final_source_ds, final_target_ds)


In [ ]:
value = 10*3.09
a = '{:.05f}'.format(value)
a

'30.90000'

In [ ]:
value = 10*3.09
a = f'{value:.02f}'
a

'30.90'

In [ ]:
value = 1009*3.09
a = f'{value:19.02f}'
a

'            3117.81'

In [ ]:
a = '{:2d}'.format(10)
a

'10'

In [ ]:
a = '{:03d}'.format(10)
a

'010'

In [ ]:
"Value: %03d" % 2

'Value: 002'

In [ ]:
"Value: %.03f" % 2

'Value: 2.000'

In [ ]:
"Value: %.03f %02d" % (2, 2)

'Value: 2.000 02'

# **Save Model**

In [ ]:
https://www.tensorflow.org/tutorials/keras/save_and_load

Les formats de fichiers .keras, .ckpt (Checkpoint), et .h5 (HDF5) sont tous utilisés pour enregistrer des modèles TensorFlow (notamment avec l'API Keras). Chacun a ses propres caractéristiques et avantages. Voici les principales différences entre ces formats de fichiers :

**.keras (SavedModel):**

Format par défaut lors de l'enregistrement d'un modèle Keras avec TensorFlow.
Contient le modèle complet, y compris l'architecture, les poids, la configuration et les informations d'entraînement.
Peut être utilisé pour le déploiement en production et pour la reprise de l'entraînement sans avoir à répéter la définition du modèle.
Fournit une structure hiérarchique qui permet de stocker plusieurs versions du modèle et des signatures de fonction pour le déploiement.<br>
**Le modèle est enrégistrer sur un fichier zip. model.save("model.keras")**<br>

**.ckpt (Checkpoint):**

Enregistre principalement les poids du modèle, pas l'architecture.
Utilisé pour sauvegarder uniquement les poids d'un modèle, ce qui permet de réutiliser les mêmes couches avec des architectures différentes.
Souvent utilisé pour la reprise de l'entraînement et la sauvegarde de points de contrôle (checkpoints) à différents stades de l'entraînement.
Vous devez spécifier l'architecture du modèle lors du chargement des poids depuis un fichier .ckpt.<br>
**Enrégistre trois fichiers. model.save_weights("model.ckpt")**<br>


**.h5 (HDF5):**

Peut enregistrer l'ensemble du modèle, y compris l'architecture, les poids et la configuration.
Facile à partager avec d'autres utilisateurs, car il contient toutes les informations nécessaires pour reconstruire et utiliser le modèle.
Utile pour enregistrer des modèles complets pour la reprise de l'entraînement ou pour des expériences de recherche.
Peut également être utilisé pour enregistrer uniquement les poids si vous le souhaitez.
En résumé, le format .keras (SavedModel) est généralement recommandé pour le déploiement en production, car il contient toutes les informations nécessaires pour le modèle, tandis que le format .ckpt est idéal pour la reprise de l'entraînement et la sauvegarde des poids. Le format .h5 est polyvalent, car il peut enregistrer l'ensemble du modèle ou uniquement les poids, ce qui le rend pratique pour diverses situations. Le choix du format dépend de l'utilisation spécifique de votre modèle et de vos besoins en matière de partage et de reprise de l'entraînement.<br>
**Enrégistre un seul fichier avec model.save("model.h5") ou model.save_weights("model.h5")**

# **Embedding**

In [ ]:
# A voir obligatoirement pour construire une couche embedding from scratch: https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup
class CustomEmbedding(keras.layers.Layer):
    def __init__(self, input_dim, output_dim, mask_zero=False, **kwargs):
        super().__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.mask_zero = mask_zero

    def build(self, input_shape):
        self.embeddings = self.add_weight(
            shape=(self.input_dim, self.output_dim),
            initializer="random_normal",
            dtype="float32",
        )

    def call(self, inputs):
        return tf.nn.embedding_lookup(self.embeddings, inputs) # voir : https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup

    def compute_mask(self, inputs, mask=None):
        if not self.mask_zero:
            return None
        return tf.not_equal(inputs, 0)


layer = CustomEmbedding(10, 32, mask_zero=True)
x = np.random.random((3, 10)) * 9
x = x.astype("int32")

y = layer(x)
mask = layer.compute_mask(x)
a = LSTM(3)(y, mask=mask)

print(mask)

params: correspond aux tensors de paramètres nombre de ligne taille du vocabulaire, nombre de colonne output_dim
ids: les ids ou tokens pour l'entrée donnée

"""
EXEMPLE:
SOIT Params = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]]
et ids = [0, 3, 4]

alors l'embedding va être calculé de la façon suivante: pour 0 on prend [1,2], pour 3 on prend [7, 8], 4 prend [9, 10]

Voir embedding avec mask = True dans https://www.tensorflow.org/guide/keras/understanding_masking_and_padding
"""

# **Réseau de neurones séquentiel**

## **LSTM**

In [ ]:
lstm = tf.keras.layers.LSTM(2, return_sequences=True, return_state=True)
o, c, h = lstm(b)

In [ ]:
o

<tf.Tensor: shape=(1, 6, 2), dtype=float32, numpy=
array([[[-0.0064925 ,  0.00696212],
        [-0.00547939,  0.0159462 ],
        [-0.0029227 ,  0.00401334],
        [-0.0029227 ,  0.00401334],
        [-0.0029227 ,  0.00401334],
        [-0.0029227 ,  0.00401334]]], dtype=float32)>

In [ ]:
c

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.0029227 ,  0.00401334]], dtype=float32)>

In [ ]:
h

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.00583435,  0.00805774]], dtype=float32)>

## **GRU**

In [ ]:
lstm = tf.keras.layers.SimpleRNN(2, return_sequences=True, return_state=True, activation="softmax")
o,  h = lstm(b)

In [ ]:
o

<tf.Tensor: shape=(1, 6, 2), dtype=float32, numpy=
array([[[0.5032558 , 0.4967442 ],
        [0.6602687 , 0.33973125],
        [0.70205325, 0.29794678],
        [0.70205325, 0.29794678],
        [0.70205325, 0.29794678],
        [0.70205325, 0.29794678]]], dtype=float32)>

In [ ]:
h

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.70205325, 0.29794678]], dtype=float32)>

## **RNN**

In [ ]:
# Output = Hidden
lstm = tf.keras.layers.SimpleRNN(2, return_sequences=True, return_state=True, activation="softmax")
o,  h = lstm(b)

In [ ]:
o

<tf.Tensor: shape=(1, 6, 2), dtype=float32, numpy=
array([[[0.5032558 , 0.4967442 ],
        [0.6602687 , 0.33973125],
        [0.70205325, 0.29794678],
        [0.70205325, 0.29794678],
        [0.70205325, 0.29794678],
        [0.70205325, 0.29794678]]], dtype=float32)>

In [ ]:
h

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.70205325, 0.29794678]], dtype=float32)>

# **Word2Vec**


L'entraînement d'un modèle Word2Vec implique l'utilisation d'un corpus de texte pour créer des représentations vectorielles des mots, de manière à capturer leurs relations sémantiques et syntaxiques. Voici une procédure générale pour entraîner un modèle Word2Vec :

Préparation du Corpus : Rassemblez un corpus de texte qui sera utilisé pour entraîner le modèle. Plus le corpus est vaste, mieux le modèle peut apprendre les relations entre les mots.

Tokenisation : Divisez le texte en unités plus petites, telles que des phrases ou des mots. Pour Word2Vec, la tokenisation est généralement effectuée au niveau des mots.

Création de Paires de Contexte-Cible : Pour chaque mot dans le corpus, créez des paires de mots qui apparaissent dans son contexte. Par exemple, pour le mot "chat", les paires de contexte-cible pourraient être ("animal", "mignon"), ("grimpe", "arbre"), etc.

Encodage One-Hot : Convertissez les mots en vecteurs one-hot. Chaque mot est représenté par un vecteur où un seul élément est défini à 1, indiquant la position du mot dans le vocabulaire.

Construction du Modèle Word2Vec : Utilisez le corpus préparé pour entraîner le modèle Word2Vec. Deux architectures couramment utilisées sont Skip-Gram et Continuous Bag of Words (CBOW). Dans Skip-Gram, le modèle prédit les mots voisins à partir du mot actuel, tandis que dans CBOW, il prédit le mot actuel à partir de ses voisins.

Entraînement du Modèle : Entraînez le modèle en ajustant les poids des vecteurs de mots pour minimiser une fonction de perte. Les algorithmes populaires pour cet entraînement incluent l'algorithme Skip-Gram négatif (Negative Sampling) et l'algorithme de Hierarchical Softmax.

Obtention des Vecteurs de Mots : Une fois l'entraînement terminé, les vecteurs de mots appris peuvent être extraits du modèle. Ces vecteurs représentent la sémantique des mots dans un espace vectoriel.

Évaluation (Facultatif) : Si des données d'évaluation sont disponibles, vous pouvez évaluer la qualité du modèle en vérifiant sa capacité à effectuer des tâches spécifiques telles que la similarité sémantique ou la complétion de mots.



La sémantique se réfère à la signification des mots, des phrases, des expressions et du langage en général.

Syntaxe : La syntaxe concerne la structure formelle d'une langue. Elle spécifie comment les mots et les phrases doivent être ordonnés pour former des énoncés corrects sur le plan grammatical. Les règles syntaxiques définissent la manière dont les éléments du langage peuvent être combinés pour créer des unités plus grandes, comme les phrases et les paragraphes.

Sémantique : La sémantique concerne la signification des éléments linguistiques. Elle explore comment les mots, les phrases et les discours transmettent des informations et des idées. La sémantique examine les relations sémantiques entre les mots, les nuances de sens, et la manière dont le contexte influence l'interprétation du langage.

Lexique : Le lexique fait référence à l'ensemble des mots d'une langue, y compris leurs significations et leurs propriétés grammaticales. C'est la collection des termes et expressions utilisés dans une langue particulière.

In [ ]:
# https://www.andreaperlato.com/theorypost/introduction-to-word2vec/
# https://colab.research.google.com/github/davidarps/2022_course_embeddings_and_transformers/blob/main/Visualizing_Attention_with_BertViz.ipynb attention
# https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb bert
# https://colab.research.google.com/github/BritneyMuller/colab-notebooks/blob/master/Easy_Text_Summarization_with_BART.ipynb#scrollTo=CdyRipC0o8vR: Text Summarisation avec BART
# https://lesdieuxducode.com/blog/2019/4/bert--le-transformer-model-qui-sentraine-et-qui-represente tout sur BERT a voir obligatoirement

from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input
from keras.preprocessing import text
from keras.preprocessing.sequence import skipgrams

# Exemple de corpus
corpus = ["le petit chat noir", "le chien marron", "la souris grise"]

# Tokenisation et création des paires de contexte-cible
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(corpus)
skip_grams = [skipgrams(sequence, vocabulary_size=total_words, window_size=2) for sequence in sequences]
data, labels = zip(*skip_grams)

In [ ]:
skip_grams

[([[2, 4],
   [4, 3],
   [2, 3],
   [1, 3],
   [3, 4],
   [1, 8],
   [3, 8],
   [3, 2],
   [3, 1],
   [1, 9],
   [2, 2],
   [2, 1],
   [4, 2],
   [1, 2],
   [4, 8],
   [2, 7],
   [4, 9],
   [3, 5],
   [2, 3],
   [3, 4]],
  [1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1]),
 ([[5, 1],
   [5, 3],
   [5, 6],
   [5, 1],
   [6, 6],
   [6, 5],
   [6, 1],
   [1, 5],
   [1, 1],
   [6, 8],
   [1, 6],
   [1, 6]],
  [0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1]),
 ([[8, 6],
   [8, 7],
   [9, 8],
   [7, 1],
   [9, 5],
   [7, 8],
   [8, 9],
   [8, 7],
   [9, 7],
   [9, 6],
   [7, 9],
   [7, 2]],
  [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0])]

In [ ]:
data

([[2, 4],
  [4, 3],
  [2, 3],
  [1, 3],
  [3, 4],
  [1, 8],
  [3, 8],
  [3, 2],
  [3, 1],
  [1, 9],
  [2, 2],
  [2, 1],
  [4, 2],
  [1, 2],
  [4, 8],
  [2, 7],
  [4, 9],
  [3, 5],
  [2, 3],
  [3, 4]],
 [[5, 1],
  [5, 3],
  [5, 6],
  [5, 1],
  [6, 6],
  [6, 5],
  [6, 1],
  [1, 5],
  [1, 1],
  [6, 8],
  [1, 6],
  [1, 6]],
 [[8, 6],
  [8, 7],
  [9, 8],
  [7, 1],
  [9, 5],
  [7, 8],
  [8, 9],
  [8, 7],
  [9, 7],
  [9, 6],
  [7, 9],
  [7, 2]])

In [ ]:
labels

([1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1],
 [0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0])

In [ ]:
a = ([ [1,2], [3,4] ], [ [5,6], [7,8] ])
c, d = zip(*a)

In [ ]:
c

([1, 2], [5, 6])

In [ ]:
d

([3, 4], [7, 8])

In [ ]:
"""
CBOW: predict target word in a given context words
SKIP-GRAMS: predict context words, given target word
see the implementation: https://radimrehurek.com/gensim/models/word2vec.html
"""

In [ ]:
sequences

[[1, 2, 3, 4], [1, 5, 6], [7, 8, 9]]

In [ ]:
tokenizer.index_word

{1: 'le',
 2: 'petit',
 3: 'chat',
 4: 'noir',
 5: 'chien',
 6: 'marron',
 7: 'la',
 8: 'souris',
 9: 'grise'}

In [ ]:
skip_grams

[([[2, 1],
   [4, 3],
   [2, 3],
   [3, 1],
   [1, 1],
   [4, 1],
   [1, 2],
   [3, 4],
   [3, 2],
   [4, 2],
   [1, 3],
   [2, 1],
   [3, 7],
   [3, 6],
   [2, 4],
   [4, 4],
   [2, 7],
   [2, 4],
   [3, 7],
   [1, 7]],
  [0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0]),
 ([[6, 1],
   [6, 2],
   [5, 1],
   [5, 6],
   [1, 6],
   [6, 5],
   [5, 3],
   [6, 6],
   [1, 5],
   [5, 1],
   [1, 1],
   [1, 3]],
  [1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0]),
 ([[7, 8],
   [8, 9],
   [8, 5],
   [7, 9],
   [9, 8],
   [9, 3],
   [8, 9],
   [7, 6],
   [9, 7],
   [8, 7],
   [7, 6],
   [9, 7]],
  [1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1])]

## **Exemple 0**

Dans cet exemple, pour chaque target, 1 contexte positif et 4 contextes négatifs sont utilisés. Les targets et contextes sont donnés en entrée. Un Embedding est utilisé pour le target et 1 Embedding pour les contextes. https://www.tensorflow.org/text/tutorials/word2vec

Le principe de cet exercice est de construire est de construire une matrice de **target**, **context** et **label**. <br>

**target** correspond aux mots cibles. On les répresente en se basant sur les ids des tokens, ça va être un tableau de taille (N, 1) ou (N, ).<br>

**context** correspond aux mots contexte, des targets. Pour cet exemple, chaque target aura 5 contextes, le premier c'est le contexte positif et les 4 suivants, les contextes négatifs. Donc un tableau de taille (N, 5).<br>

**label** c'est un tableau de taille (N, 5) avec chaque vecteur sous la forme (1, 0, 0, 0, 0) avec 1 pour dire contexte positif et 0 pour dire contexte négatif.<br>

Les contextes positifs sont construits avec **tf.keras.preprocessing.skipgrams** et les contextes négatifs par **tf.random.log_uniform_candidate_sampler**<br>


Le modèle est entrainé en utilisant deux Embedding, car prenant deux entrées. Le premier Embedding pour le target et le deuxième pour context. tf.einsum est utilisé pour multiplié le codage du target sur le codage du context.

In [ ]:
# Construire contextes positif:

# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)

# Construire pair positif:
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0 # 0 pour dire qu'on ne prend pas de pair négatif
      )


In [ ]:
import tensorflow as tf

# Création d'un tenseur avec une dimension de taille 1
tensor_with_singleton_dim = tf.constant([[[1]], [[2]], [[3]]])

# Utilisation de tf.squeeze pour supprimer la dimension de taille 1
squeezed_tensor = tf.squeeze(tensor_with_singleton_dim, axis=(1,2)) # élimine l'axe 1 et 2

# Affichage des formes avant et après l'opération
print("Avant squeeze :", tensor_with_singleton_dim.shape)
print("Après squeeze :", squeezed_tensor.shape)


Avant squeeze : (3, 1, 1)
Après squeeze : (3,)


In [ ]:
p, l = tf.keras.preprocessing.sequence.skipgrams(
      [2,3,4,0,0,0],
      vocabulary_size=5,
      window_size=2,
      negative_samples=0)
p

[[3, 2], [4, 3], [2, 4], [2, 3], [3, 4], [4, 2]]

In [ ]:
l

[1, 1, 1, 1, 1, 1]

In [ ]:
# https://www.tensorflow.org/text/tutorials/word2vec
import tensorflow as tf
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

with open(path_to_file) as f:
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)


text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))


1115394/1115394 [==============================] - 0s 0us/step
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [ ]:
for i in text_ds.take(10):
    print(i)

tf.Tensor(b'First Citizen:', shape=(), dtype=string)
tf.Tensor(b'Before we proceed any further, hear me speak.', shape=(), dtype=string)
tf.Tensor(b'All:', shape=(), dtype=string)
tf.Tensor(b'Speak, speak.', shape=(), dtype=string)
tf.Tensor(b'First Citizen:', shape=(), dtype=string)
tf.Tensor(b'You are all resolved rather to die than to famish?', shape=(), dtype=string)
tf.Tensor(b'All:', shape=(), dtype=string)
tf.Tensor(b'Resolved. resolved.', shape=(), dtype=string)
tf.Tensor(b'First Citizen:', shape=(), dtype=string)
tf.Tensor(b'First, you know Caius Marcius is chief enemy to the people.', shape=(), dtype=string)


In [ ]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
import re
import string
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)


In [ ]:
vectorize_layer.adapt(text_ds.batch(1024))

In [ ]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [ ]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(tf.data.AUTOTUNE).map(vectorize_layer).unbatch()

In [ ]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

32777


In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
import tqdm
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=242)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")


100%|██████████| 32777/32777 [00:43<00:00, 761.86it/s] 




targets.shape: (64702,)
contexts.shape: (64702, 5)
labels.shape: (64702, 5)


In [ ]:
targets

array([125, 125, 125, ...,  82,  82,  82])

In [ ]:
targets.shape

(64702,)

In [ ]:
contexts

array([[ 673,   69,    5,   57, 2119],
       [ 144, 1170,   95,    1,   34],
       [  16, 1057,  581,  108,    8],
       ...,
       [   3,  252,   13,  491,  536],
       [  28,   40,    4,   61,  753],
       [ 674,  133,  244, 3461,   60]])

In [ ]:
contexts.shape

(64702, 5)

In [ ]:
# Le premier mot dans contexts est le mot qui se trouve sur le target et les 4 mots qui reste ne sont pas dans le contexte donc faux d'où 0.
labels

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]])

In [ ]:
labels.sum(axis=0)

array([64702,     0,     0,     0,     0])

In [ ]:
labels.shape

(64702, 5)

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)


<_BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [ ]:
dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
print(dataset)


<_PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [ ]:
# Set the number of negative samples per positive context.
num_ns = 4

class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = tf.keras.layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = tf.keras.layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb) # b: batch, c: sequence = 5, e: embedding_dim
    # dots: (batch, context)
    return dots

In [ ]:
target_embedding = tf.keras.layers.Embedding(5,
                                  5,
                                  input_length=1,
                                  name="w2v_embedding")

In [ ]:
target_embedding(np.array([2]))

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[ 0.01872643,  0.04774718, -0.03377201, -0.0115474 , -0.02404326]],
      dtype=float32)>

In [ ]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])


In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")


In [ ]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])


In [ ]:
#docs_infra: no_execute
%tensorboard --logdir logs

In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
index = 3
vec = weights[index]

## **Example 1: TimeDistributed et einsum**

TimeDistributed enveloppe une couche Dense dans cet exemple pour traiter l'entrée (la séquence) de manière indépendante, c'est à dire chaque timestamp est considéré comme un vecteur d'entrée. Si on ne l'utilise pas, la couche Dense considère toute la séquence comme un vecteur simple donc supprime l'ordre.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, RepeatVector, TimeDistributed

# Création du modèle séquentiel
model = Sequential()

# Ajout d'une couche Dense pour traiter le vecteur d'entrée de dimension 1
model.add(Dense(8, input_dim=1, activation='relu'))

# Ajout d'une couche RepeatVector pour répéter le vecteur de sortie
model.add(RepeatVector(2))

# Ajout d'une couche TimeDistributed pour appliquer la couche Dense à chaque pas de temps de la séquence au lieu de considérer la séquence comme un vecteur
model.add(TimeDistributed(Dense(4, activation='softmax')))

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Affichage de la structure du modèle
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 16        
                                                                 
 repeat_vector (RepeatVecto  (None, 2, 8)              0         
 r)                                                              
                                                                 
 time_distributed (TimeDist  (None, 2, 4)              36        
 ributed)                                                        
                                                                 
Total params: 52 (208.00 Byte)
Trainable params: 52 (208.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, RepeatVector, TimeDistributed

# Création du modèle séquentiel
model = Sequential()

# Ajout d'une couche Dense pour traiter le vecteur d'entrée de dimension 1
model.add(Dense(8, input_dim=1, activation='relu'))

# Ajout d'une couche RepeatVector pour répéter le vecteur de sortie
model.add(RepeatVector(2))

# Ajout d'une couche TimeDistributed pour appliquer la couche Dense à chaque pas de temps de la séquence
model.add(Dense(4, activation='softmax'))

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Affichage de la structure du modèle
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 8)                 16        
                                                                 
 repeat_vector_1 (RepeatVec  (None, 2, 8)              0         
 tor)                                                            
                                                                 
 dense_3 (Dense)             (None, 2, 4)              36        
                                                                 
Total params: 52 (208.00 Byte)
Trainable params: 52 (208.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import tensorflow as tf

# Tenseurs d'exemple
a = tf.constant([[1, 2], [3, 4]])
a

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>

In [ ]:
b = tf.constant([[[5, 6], [7, 8], [9, 10]], [[11, 12], [13, 14], [15, 16]]])
b

<tf.Tensor: shape=(2, 3, 2), dtype=int32, numpy=
array([[[ 5,  6],
        [ 7,  8],
        [ 9, 10]],

       [[11, 12],
        [13, 14],
        [15, 16]]], dtype=int32)>

In [ ]:

# Opération einsum
result = tf.einsum('be,bce->bc', a, b)

# Affichage du résultat
print(result.numpy())


[[ 17  23  29]
 [ 81  95 109]]


In [ ]:
import re
import string
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int')


In [ ]:
data = ["I am very pround", "you are here"]

vectorize_layer.adapt(data)

In [ ]:
vectorize_layer.get_vocabulary()

['', '[UNK]', 'you', 'very', 'pround', 'i', 'here', 'are', 'am']

In [ ]:
vectorize_layer(data)

<tf.Tensor: shape=(2, 4), dtype=int64, numpy=
array([[5, 8, 3, 4],
       [2, 7, 6, 0]])>

# **Attention**

**Dot product Attention**

Pour deux vecteurs $h_i$ et $h_j$, le produit scalaire donne le poids d'attention. on considère que $h$ est un vecteur colonne :<br><br>
$score  = h_i^T.h_j$<br><br>
$AttentionDot(h_i, h_j) = \frac{\exp(score(h_i, h_j))}{\sum_k \exp(score(h_i, h_k))}$

In [ ]:
import tensorflow as tf

class DotProductAttention(tf.keras.layers.Layer):
    def __init__(self):
        super(DotProductAttention, self).__init__()

    def call(self, query, keys, values):
        """
          Trois réseaux doivent être utilisés chacun de units = dim, qui produisent query, keys et values
          query: (batch, Tq, dim)
          keys: (batch, Tk, dim)
          values: (batch, Tv, dim)
        """
        # Calcul du produit scalaire entre la requête et les clés
        scores = tf.matmul(query, keys, transpose_b=True) # (batch, Tq, Tk)

        # Calcul des poids d'attention avec la fonction softmax
        attention_weights = tf.nn.softmax(scores, axis=-1) # (batch, Tq, Tk)

        # Calcul de la somme pondérée des valeurs
        context_vector = tf.matmul(attention_weights, values) # (batch, Tq, dim)

        return context_vector, attention_weights

# Exemple d'utilisation
attention_layer = DotProductAttention()

# Création d'une requête, de clés et de valeurs fictives
query = tf.random.normal([1, 32])
keys = tf.random.normal([1, 10, 32])
values = tf.random.normal([1, 10, 64])

# Calcul de l'attention
context_vector, attention_weights = attention_layer(query, keys, values)

print("Query:", query)
print("Keys:", keys)
print("Values:", values)
print("Attention Weights:", attention_weights)
print("Context Vector:", context_vector)


**Luong Attention**

Pour deux vecteurs $h_i$ et $h_j$, le produit scalaire donne le poids d'attention :<br><br>
$score  = h_i.W_a.h_j$<br><br>
$W_a$ matrices de poids (plusieurs neurones = units=n)<br><br>
$AttentionLuong(h_i, h_j) = \frac{\exp(score(h_i, h_j))}{\sum_k \exp(score(h_i, h_k))}$

In [ ]:
import tensorflow as tf

class LuongAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(LuongAttention, self).__init__()
        self.W = tf.keras.layers.Dense(units)

    def call(self, query, keys):
        """
        query: (batch, dim) souvent le hidden state du decoder (dim=units)
        keys: (batch, Tk, dim)
        """
        # Ajout d'une nouvelle dimension à la requête pour permettre la concaténation
        query_with_time_axis = tf.expand_dims(query, 1) # (batch, 1, dim)

        # Calcul des scores d'attention
        score = tf.matmul(query_with_time_axis, self.W(keys), transpose_b=True) # (batch, 1, Tk)

        # Calcul des poids d'attention avec la fonction softmax
        attention_weights = tf.nn.softmax(score, axis=2) # (batch, 1, Tk)

        # Calcul de la somme pondérée des valeurs
        context_vector = tf.matmul(attention_weights, keys) # (batch, 1, dim)

        return tf.squeeze(context_vector, axis=1), attention_weights # (batch, dim), (batch, 1, Tk)

# Exemple d'utilisation
units = 32
attention_layer = LuongAttention(units)

# Création d'une requête et de clés fictives
query = tf.random.normal([1, units])
keys = tf.random.normal([1, 10, units])

# Calcul de l'attention
context_vector, attention_weights = attention_layer(query, keys)

print("Query:", query)
print("Keys:", keys)
print("Attention Weights:", attention_weights)
print("Context Vector:", context_vector)


**Bahdanau Attention (Additive)**

Pour deux vecteurs $h_i$ et $h_j$, le produit scalaire donne le poids d'attention :<br><br>
$score  = v_a^T.tanh(W_{a1}.h_i + W_{a2}.h_j)$<br><br>
$W_{a1}, W_{a2}$ matrices de poids (plusieurs neurones, units=n) <br><br>
$v_a$ vecteur de poids (1 neurone = units=1)<br><br>
$AttentionBahdanau(h_i, h_j) = \frac{\exp(score(h_i, h_j))}{\sum_k \exp(score(h_i, h_k))}$

In [ ]:
import tensorflow as tf

class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        """
        query: (batch, dim) souvent le hidden state du decoder
        values: (batch, Tv, dim)
        """
        # Ajout d'une nouvelle dimension à la requête pour permettre la concaténation
        query_with_time_axis = tf.expand_dims(query, 1) # (batch, 1, dim)

        # Calcul des scores d'attention
        score = tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)) # (batch, Tv, dim)

        # Calcul des poids d'attention
        attention_weights = tf.nn.softmax(self.V(score), axis=1) # (batch, Tv, 1)

        # Calcul de la somme pondérée des valeurs
        context_vector = attention_weights * values # (batch, Tv, dim)
        context_vector = tf.reduce_sum(context_vector, axis=1) # (batch, dim)

        return context_vector, attention_weights

# Exemple d'utilisation
units = 32
attention_layer = BahdanauAttention(units)

# Création d'une requête et de valeurs fictives
query = tf.random.normal([1, units])
values = tf.random.normal([1, 10, units])

# Calcul de l'attention
context_vector, attention_weights = attention_layer(query, values)

print("Query:", query)
print("Values:", values)
print("Attention Weights:", attention_weights)
print("Context Vector:", context_vector)


$Attention(Q,K,V)=softmax(Q⋅W_q+K⋅W_k)⋅V$

Etapes algorithmes: <br>
1. Expands query si nécessaire<br>
2. Calculer scores<br>
3. caculer attention<br>
4. Faire matmul (dot et Luong) ou * (Bahdanau) de attention avec values.<br>
5. Retourner sortie pour dot (sequence), retourner squeeze(sortie, axis=1) pour Luong (vecteur) et reduce_sum(sortie, axis=1) (vecteur).<br>